In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import crocoddyl as c
import numdifftools as nd
from time import perf_counter
c.switchToNumpyArray()

In [3]:
# get the residual data
positions = []
residuals = []
model = c.ActionModelUnicycle()


for _ in range(1000):
    
    x0 = np.array([np.random.uniform(-2.1, 2.1), np.random.uniform(-2.1, 2.1), np.random.uniform(0,1)])
    T = 30
    problem = c.ShootingProblem(x0.T, [ model ] * T, model)
    ddp = c.SolverDDP(problem)
    ddp.solve()

    for d in ddp.datas():
        residual = d.r
    positions.append(x0)
    residuals.append(residual)

positions = np.asarray(positions)
residuals = np.asarray(residuals)
del model

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')

x_train = torch.as_tensor(positions, device = device, dtype = torch.float32)
y_train = torch.as_tensor(residuals, device = device, dtype = torch.float32)
print(y_train.size())

torch.Size([1000, 5])


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(3, 8)
        self.fc2 = nn.Linear(8, 8)
        self.fc3 = nn.Linear(8, 5)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net = net.float()
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
       values taken from a normal distribution.'''

    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        y = m.in_features
    # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0.0,1/np.sqrt(y))
    # m.bias.data should be 0
        m.bias.data.fill_(0)

net.apply(weights_init_normal)

Net(
  (fc1): Linear(in_features=3, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=5, bias=True)
)

In [6]:
for epoch in range(100):
    #total_loss = 0
    for inputs, target in zip(x_train, y_train):   
        optimizer.zero_grad()
        y_hat = net(x_train.float())
        loss = criterion(y_hat, y_train)
        loss.backward()
        optimizer.step()
        

In [9]:
def a2m(a):
    return np.matrix(a).T

def m2a(m):
    return np.array(m).squeeze()


c.switchToNumpyMatrix()

class UnicycleTerminal(c.ActionModelAbstract):
    def __init__(self, net):
        c.ActionModelAbstract.__init__(self, c.StateVector(3), 2, 5)
        self.net = net
                
        self.w1 = net.fc1.weight.data.cpu().numpy()
        self.b1 = net.fc1.bias.data.cpu().numpy()

        self.w2 = net.fc2.weight.data.cpu().numpy()
        self.b2 = net.fc2.bias.data.cpu().numpy()

        self.w3 = net.fc3.weight.data.cpu().numpy()
        self.b3 = net.fc3.bias.data.cpu().numpy()

        
        
    def calc(self, data, x, u=None):        
        x0 = torch.as_tensor(x.reshape(1, -1), device = device, dtype = torch.float32)
        prediction = net(x0).view(5, 1)
        r_ = prediction.cpu()
        data.rr = r_.detach().numpy()
        data.cost = 0.5 * float(torch.matmul(prediction.T, prediction))
                   
        
    def calcDiff(self, data, x, u=None, recalc=True):
        if u is None:
            u = self.unone
        if recalc:
            self.calc(data, x, u)
        
        x0 = m2a(x)

        def residual(y):
            
            out1 = np.tanh(y.dot(self.w1.T) + self.b1)
            out2 = np.tanh(out1.dot(self.w2.T) + self.b2)
            residual = out2.dot(self.w3.T) + self.b3

            return residual[0:3]

        j = nd.Jacobian(residual)
        
        data.Lx = a2m(j(x0.T) @ residual(x0))
        data.Lxx = a2m(j(x0.T) @ j(x0))

In [12]:
model = c.ActionModelUnicycle()
terminal_model = UnicycleTerminal(net)

start = perf_counter()
for _ in range(1000):
    x0 = np.array([np.random.uniform(-2.1, 2.1), np.random.uniform(-2.1, 2.1), np.random.uniform(0,1)])
    T = 30
    problem = c.ShootingProblem(x0.T, [ model ] * T, terminal_model)
    ddp = c.SolverDDP(problem)
    ddp.solve([], [], 1000)
    
end = perf_counter()

KeyboardInterrupt: 

In [ ]:
print(end - start)